<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget -N https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2024-09-02 03:33:09--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘concrete_data_week4.zip’ not modified on server. Omitting download.



In [2]:
!unzip -oq concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

In [3]:
!pip install Keras-Applications
!pip install tensorflow==2.8.0

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [4]:
from keras.preprocessing.image import ImageDataGenerator

/Users/bolatanibrahim/.local/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [6]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [7]:
num_classes = 2
image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [8]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [9]:
train_generator = data_generator.flow_from_directory(
    './concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    class_mode='categorical',
    batch_size=batch_size_training
)

Found 30001 images belonging to 2 classes.


In [10]:
validation_generator = data_generator.flow_from_directory(
    './concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical'
)

Found 9501 images belonging to 2 classes.


In [11]:
model = Sequential()

2024-09-02 03:34:21.141304: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.add(
    VGG16(
        include_top=False,
        pooling='avg',
        weights='imagenet'
    )
)

In [13]:
model.add(Dense(num_classes, activation='softmax'))
model.layers

In [14]:
model.layers[0].layers

In [15]:
model.layers[0].trainable = False

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [19]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1
)

Epoch 1/2
301/301 [==============================] - 5755s 19s/step - loss: 0.0760 - accuracy: 0.9752 - val_loss: 0.0232 - val_accuracy: 0.9948
Epoch 2/2
301/301 [==============================] - 6559s 22s/step - loss: 0.0175 - accuracy: 0.9955 - val_loss: 0.0136 - val_accuracy: 0.9967


In [20]:
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [22]:
resnet_model = keras.models.load_model('./classifier_resnet_model.h5')
vgg16_model = keras.models.load_model('./classifier_vgg16_model.h5')

In [23]:
test_generator = data_generator.flow_from_directory(
    './concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False
)

Found 500 images belonging to 2 classes.


In [24]:
resnet_evaluation_history = resnet_model.evaluate(test_generator)
vgg16_evaluation_history = vgg16_model.evaluate(test_generator)

16/16 [==============================] - 60s 4s/step - loss: 0.0103 - accuracy: 0.9960


In [25]:
print('VGG16 performance: ', vgg16_evaluation_history)
print('ResNet performance: ', resnet_evaluation_history)

VGG16 performance:  [0.010298829525709152, 0.9959999918937683]
ResNet performance:  [0.008076358586549759, 0.9959999918937683]


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [28]:
import numpy as np

In [34]:
resnet_predictions = resnet_model.predict(test_generator)
resnet_round_preds = np.around(resnet_predictions)
ressnet_class_preds = np.argmax(resnet_predictions, axis=-1)

In [35]:
vgg16_predictions = vgg16_model.predict(test_generator)
vgg16_round_preds = np.around(vgg16_predictions)
vgg16_class_preds = np.argmax(vgg16_predictions, axis=-1)

In [31]:
def convert_predict_to_classes(predictions, sample_size=5):
    for negative, positive in predictions[0:sample_size]:
        print('Negative' if negative == 1 else 'Positive')

In [39]:
print('ResNet prediction for the test set........')
convert_predict_to_classes(resnet_round_preds)

ResNet prediction for the test set........
Negative
Negative
Negative
Negative
Negative


In [40]:
print('VGG16 prediction for the test set........')
convert_predict_to_classes(vgg16_round_preds)

VGG16 prediction for the test set........
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).